In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tf.enable_eager_execution()

In [ ]:
import altair as alt
import pandas as pd
%matplotlib inline

alt.renderers.enable("notebook")

In [ ]:
alpha = 2.
beta = 5.
gamma = tfp.distributions.Gamma(concentration=alpha, rate=beta)
lognormal = tfp.distributions.LogNormal(loc=-1.6, scale=1.1)

x = np.linspace(0.01, 0.6)
df = pd.DataFrame({"x": x, "gamma": gamma.prob(x).numpy(), "lognormal": lognormal.prob(x).numpy()})
df.plot(x="x", y=["gamma", "lognormal"], kind="line")


In [ ]:
def tf_lognormal_multi_elbo_grad(loc, shape, true_distribution, particle_count = 100):
    # A variety of epsilons.
    epsilon = tf.constant(np.random.normal(0., 1., particle_count).astype(np.float32))
    with tf.GradientTape() as g:
        tf_loc = tf.constant(loc)
        tf_scale = tf.constant(shape)
        g.watch(tf_loc)
        g.watch(tf_scale)
        tf_x = tf.math.exp(tf_loc + tf_scale * epsilon)
        # This is the log of the full sum of ratios as in the equation just before (7)
        # in the 2018 ICLR paper.
        y = tf.math.log(
            # In principle we should have a 1/K term here, but it disappears in the log
            # grad.
            tf.math.reduce_sum(
                true_distribution.prob(tf_x)
                / tfp.distributions.LogNormal(loc=tf_loc, scale=tf_scale).prob(tf_x)
            )
        )
        x_arr = np.array([tf_x.numpy()]).transpose()
        tf_gradient = np.array([grad.numpy() for grad in g.gradient(y, [tf_loc, tf_scale])])
    return tf_gradient

def gradient_analysis(particle_count, gradient_count):
    gradients = np.concatenate([
        np.array([tf_lognormal_multi_elbo_grad(-1.6, 1.1, gamma, particle_count)]) for _ in range(gradient_count)])
    gradients_df = pd.DataFrame(gradients, columns=["loc_grad", "scale_grad"])

    print(gradients_df.describe())
    return alt.Chart(gradients_df).mark_bar().encode(
        alt.X("loc_grad", bin=alt.Bin(step=0.05)),
        y='count()',
    )

In [ ]:
gradient_analysis(particle_count = 100, gradient_count = 1000)

In [ ]:
gradient_analysis(particle_count = 1000, gradient_count = 1000)